In [2]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-eqgp4llf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-eqgp4llf
  Resolved https://github.com/huggingface/transformers to commit 2b8068c306a4c79350e1af5fea5f7e3d93d82d95
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

# default: Load the model on the available device(s)
model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-4B-Instruct",
    dtype="auto",
    device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen3VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen3-VL-4B-Instruct",
#     dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-4B-Instruct")

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [4]:
def inference(image, prompt):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image,
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]
    
    # Preparation for inference
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    )
    inputs = inputs.to(model.device)
    
    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=1280)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    for text in output_text:
        print(text)

In [5]:
from PIL import Image

def resize_image(image_path, height, width):
    new_size = (height, width)
    image = Image.open(image_path)
    resized_image = image.resize(new_size, Image.LANCZOS)
    return resized_image 

### 1. No Preprocessing, Image resized, markdown

In [8]:
image = '/kaggle/input/psc-forms/20251111_160053.jpg'
resized_image = resize_image(image, 1024, 1024)
prompt = 'Convert to markdown'
inference(resized_image, prompt)

```markdown
REPUBLIC OF MAURITIUS
PUBLIC SERVICE COMMISSION
Application Form

P.S.C. Form 7
FOR OFFICIAL USE

Read carefully the enclosed Notes
& Instructions to candidates
before filling this form

1. Post Applied for... Administrative Officer
Ministry/Department... Ministry of Finance
Date of Advertisement/Vacancy Circular... VS/US/2225
Previous Posts Applied for
1. Management Analyst.
2. Executive Assistant.

2. National Identity No.
Title Mr □ Mrs □ Miss □ Ms □
Marital Status : Married □ Single □ Other : ……………………
Surname …………………… (in block letters)
Other Names : Silverstone (in block letters)
Maiden Name (if applicable) : Not Applicable

3. Residential Address :
No.15, FLOWER LANE
(Clombe. 07) (in block letters)

Phone No. Office. 01234567. Home. 07712345 Mobile. 57712345. Email address : Scarlett@gmail.com
Date of Birth 15.08.1915 Age 301 Place of Birth. Clombe.
Nationality. American Certificate No. (if Naturalised)… N/A …& Date. 11.11.2025

4. SECONDARY ORDINARY LEVEL
State wheth

### No Preprocessing, Image resized, Json

In [7]:
image = '/kaggle/input/psc-forms/20251111_160053.jpg'
resized_image = resize_image(image, 1024, 1024)
prompt = """
Extract all visible text and structured content from this image or PDF.
Do not omit any textual element, even if faint or partially visible.
If handwriting~ or unclear text appears, write [unreadable] or your best guess in brackets.
{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "title": "PSCApplication",
  "type": "object",
  "properties": {
    "post_applied_for": { "type": "string" },
    "ministry_department": { "type": "string" },
    "date_of_advertisement": { "type": "string", "format": "date" },
    "national_identity_no": { "type": "string" },
    "surname": { "type": "string" },
    "other_names": { "type": "string" },
    "residential_address": { "type": "string" },
    "date_of_birth": { "type": "string", "format": "date" },
    "age": { "type": "integer" },
    "place_of_birth": { "type": "string" },
    "nationality": { "type": "string" },
    "phone_mobile": { "type": "string" },
    "email": { "type": "string", "format": "email" },
 
    "degree_qualifications": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "level": { "type": "string" },
          "institution": { "type": "string" },
          "country": { "type": "string" },
          "qualification_name": { "type": "string" },
          "class_division_level": { "type": "string" },
          "date_of_result": { "type": "string", "format": "date" },
          "course_type": { "type": "string" },
          "duration_from": { "type": "string", "format": "date" },
          "duration_to": { "type": "string", "format": "date" }
        },
        "required": [
          "level", "institution", "country", "qualification_name",
          "class_division_level", "date_of_result", "course_type",
          "duration_from", "duration_to"
        ]
      }
    },
 
    "diploma_qualifications": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "level": { "type": "string" },
          "institution": { "type": "string" },
          "country": { "type": "string" },
          "qualification_name": { "type": "string" },
          "class_division_level": { "type": "string" },
          "date_of_result": { "type": "string", "format": "date" },
          "course_type": { "type": "string" },
          "duration_from": { "type": "string", "format": "date" },
          "duration_to": { "type": "string", "format": "date" }
        },
        "required": [
          "level", "institution", "country", "qualification_name",
          "class_division_level", "date_of_result", "course_type",
          "duration_from", "duration_to"
        ]
      }
    },
 
    "ordinary_level_exams": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "exam_type": { "type": "string" },
          "month_year": { "type": "string" },
          "index_no": { "type": "string" },
          "exam_centre_no": { "type": "string" },
          "subjects": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "subject": { "type": "string" },
                "grade": { "type": "string" }
              },
              "required": ["subject", "grade"]
            }
          },
          "result": { "type": "string" },
          "aggregate": { "type": "string" }
        },
        "required": [
          "exam_type", "month_year", "index_no",
          "exam_centre_no", "subjects", "result", "aggregate"
        ]
      }
    },
 
    "advanced_level_exams": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "exam_type": { "type": "string" },
          "month_year": { "type": "string" },
          "index_no": { "type": "string" },
          "exam_centre_no": { "type": "string" },
          "subjects": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "subject": { "type": "string" },
                "grade": { "type": "string" },
                "level": { "type": "string" }
              },
              "required": ["subject", "grade", "level"]
            }
          },
          "result": { "type": "string" },
          "aggregate": { "type": "string" }
        },
        "required": [
          "exam_type", "month_year", "index_no",
          "exam_centre_no", "subjects", "result", "aggregate"
        ]
      }
    },
 
    "current_government_employment": {
      "type": "object",
      "properties": {
        "post_held": { "type": "string" },
        "temporary_substantive": { "type": "string" },
        "ministry_department": { "type": "string" },
        "date_of_appointment": { "type": "string", "format": "date" },
        "date_of_confirmation": { "type": "string", "format": "date" },
        "present_salary": { "type": "number" }
      },
      "required": [
        "post_held", "temporary_substantive", "ministry_department",
        "date_of_appointment", "date_of_confirmation", "present_salary"
      ]
    },
 
    "other_employment": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "post_held": { "type": "string" },
          "ministry_department": { "type": "string" },
          "date_of_appointment": { "type": "string", "format": "date" },
          "present_salary": { "type": "number" }
        },
        "required": ["post_held", "ministry_department", "date_of_appointment", "present_salary"]
      }
    },
 
    "investigation_enquiry": { "type": "boolean" },
    "investigation_details": { "type": ["string", "null"] },
    "court_conviction": { "type": "boolean" },
    "conviction_details": { "type": ["string", "null"] },
    "resigned_retired_dismissed": { "type": "boolean" },
    "resignation_details": { "type": ["string", "null"] },
 
    "other_qualifications": {
      "type": "array",
      "items": { "type": "string" }
    }
  },
  "required": [
    "post_applied_for",
    "ministry_department",
    "date_of_advertisement",
    "national_identity_no",
    "surname",
    "other_names",
    "residential_address",
    "date_of_birth",
    "age",
    "place_of_birth",
    "nationality",
    "phone_mobile",
    "email"
  ]
}"""

inference(resized_image, prompt)

{
  "post_applied_for": "Administrative Officer",
  "ministry_department": "Ministry of Finance",
  "date_of_advertisement": "2025-04-05",
  "national_identity_no": "G815123456789",
  "surname": "Scarlett",
  "other_names": "Silverstone",
  "residential_address": "NO.15, FLOWER LANE, CLONBO. 07",
  "date_of_birth": "1985-08-15",
  "age": 39,
  "place_of_birth": "Clonbo",
  "nationality": "American",
  "phone_mobile": "5772346",
  "email": "Scarlett@gmail.com",
  "ordinary_level_exams": [
    {
      "exam_type": "Cambridge S.C.",
      "month_year": "2011",
      "index_no": "110123",
      "exam_centre_no": "110112",
      "subjects": [
        {
          "subject": "Mathematics",
          "grade": "A"
        },
        {
          "subject": "Science",
          "grade": "A"
        },
        {
          "subject": "English Language",
          "grade": "A"
        },
        {
          "subject": "Computer Science",
          "grade": "A"
        },
        {
          "subject